Resources
* https://www.analyticsvidhya.com/blog/2018/03/essentials-of-deep-learning-sequence-to-sequence-modelling-with-attention-part-i/
* https://www.tensorflow.org/tutorials/text/nmt_with_attention
* https://towardsdatascience.com/nlp-for-beginners-cleaning-preprocessing-text-data-ae8e306bef0f
* http://ai.baidu.com/broad/download CCMT 2019
* http://opus.nlpl.eu
* https://www.youtube.com/watch?v=oiNFCbD_4Tk


In [1]:
# from google.colab import drive
# drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [19]:
# from os import listdir, makedirs
# from os.path import isfile, join, exists
# import json
# import csv

# # array of json file names
# json_dir = './training_data'
# csv_dir = './training_data/csv'
# def get_file_names(mypath):
#   return [f for f in listdir(mypath) if isfile(join(mypath, f)) and len(f.split('.')) == 2 and f.split('.')[1] == 'json']

# # save the english and chinese translation pairs
# def get_translation(files): # get_file_names(json_dir)
#   translations = []
#   for file in files:
#     with open(f'{json_dir}/{file}', 'r') as file:
#       # translations for this file
#       for cnt, line in enumerate(file):
#         single_translation = []
#         obj = json.loads(line)
#         single_translation.append(obj['translation'])
#         single_translation.append(obj['transcript'])
#         translations.append(single_translation)
#         pass
#       pass
#     pass
#   return translations
#   pass

# # delete file if file exists
# def save_to_file(translations, dir): # get_translation(get_file_names(json_dir)), csv_dir
#   if not exists(dir):
#     makedirs(dir)
#   csv_file_dir = f'{dir}/translations.csv'
#   with open(csv_file_dir, 'w') as file:
#     writer = csv.writer(file)
#     writer.writerow(['English', 'Chinese'])
#     for row in translations:
#       writer.writerow(row)
#       pass
#     pass
#   pass

# # save_to_file(get_translation(get_file_names(json_dir)), csv_dir)
# print(len(open(f'{csv_dir}/translations.csv').readlines()))

97


In [25]:
import tensorflow as tf
import pandas as pd
import re
import sys
import os
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import time
import nltk
import string
import codecs
from sklearn.model_selection import train_test_split
!{sys.executable} -m pip install jieba
import jieba as jieba
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hartonotjakrawinata/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hartonotjakrawinata/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [35]:
# load file into dataframe
json_dir = './training_data'
csv_dir = './training_data/csv'
df = []
with open(f'{csv_dir}/translations_single.csv', 'r') as file:
  df = pd.read_csv(file)
df.head()

,English,Chinese
0,"First of all, I'd like to extend my gratitude ...",首先呢，非常感谢这个主办方，能够把一个这个亡命之徒放在第一个做坦白交待。
1,"At the very beginning, I'd like to introduce t...",然后在康开始的时候，我先给大家介绍一下，就是我们这个亡命之徒的同伙。
2,"There are men, women, teenagers, and even peop...",呃在我们亡命之徒的这个同伙里面有男人，也有女人，然后也有孩子，甚至还有这个病毒的感染者。
3,And some of them are not human beings at all.,呃还有一众亡命之徒它并不是人。
4,The daredevils are very adventurous as well as...,然后亡命之徒呢总是很喜欢冒险，然后他们偶尔也比较喜欢好斗。


In [36]:
# tokenize based on spaces, remove punctuation (english) and chinese
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
df['English'] = df['English'].apply(lambda x: tokenizer.tokenize(x.lower()))
df['Chinese'] = df['Chinese'].apply(lambda x: [w for w in jieba.cut_for_search(x)])
df.head()

# ‘\w+|\$[\d\.]+|\S+’ -> splits up by spaces or by periods that are not attached to a digit
# ‘\s+’, gaps=True -> grabs everything except spaces as a token

,English,Chinese
0,"[first, of, all, i, d, like, to, extend, my, g...","[首先, 呢, ，, 非常, 感谢, 非常感谢, 这个, 主办, 主办方, ，, 能够, 把..."
1,"[at, the, very, beginning, i, d, like, to, int...","[然后, 在, 康, 开始, 的, 时候, ，, 我, 先, 给, 大家, 介绍, 一下, ..."
2,"[there, are, men, women, teenagers, and, even,...","[呃, 在, 我们, 亡命, 亡命之徒, 的, 这个, 同伙, 里面, 有, 男人, ，, ..."
3,"[and, some, of, them, are, not, human, beings,...","[呃, 还有, 一众, 亡命, 亡命之徒, 它, 并, 不是, 人, 。]"
4,"[the, daredevils, are, very, adventurous, as, ...","[然后, 亡命, 亡命之徒, 呢, 总是, 很, 喜欢, 冒险, ，, 然后, 他们, 偶尔..."


In [37]:
# # remove stopwords (not sure how useful it will be)

# def remove_english_stopwords(text):
#   return [w for w in text if w not in nltk.corpus.stopwords.words('english')]

# chinese_stopwords = codecs.open(f'{csv_dir}/chinese_stopwords.csv','r','utf-8').read().split(',')
# def remove_chinese_stopwords(text):
#   return [w for w in text if w not in chinese_stopwords]

# df['English'] = df['English'].apply(lambda x: remove_english_stopwords(x))
# df['Chinese'] = df['Chinese'].apply(lambda x: remove_chinese_stopwords(x))
# df.head()

In [38]:
# # lemmatization and stemming
# lemmatizer = nltk.stem.WordNetLemmatizer()
# def word_lemmatizer(text):
#   return [lemmatizer.lemmatize(i) for i in text]

# df['English'] = df['English'].apply(lambda x: word_lemmatizer(x))
# df.head()

stemmer = nltk.stem.porter.PorterStemmer()
def stem_text(text):
  return [stemmer.stem(i) for i in text]

df['English'] = df['English'].apply(lambda x: stem_text(x))

def array_to_string(text):
  return '<start> ' + ' '.join(text) + ' <end>'

df['English'] = df['English'].apply(lambda x: array_to_string(x))
df['Chinese'] = df['Chinese'].apply(lambda x: array_to_string(x))
df.head()

,English,Chinese
0,<start> first of all i d like to extend my gra...,<start> 首先 呢 ， 非常 感谢 非常感谢 这个 主办 主办方 ， 能够 把 一个 ...
1,<start> at the veri begin i d like to introduc...,<start> 然后 在 康 开始 的 时候 ， 我 先 给 大家 介绍 一下 ， 就是 我...
2,<start> there are men women teenag and even pe...,<start> 呃 在 我们 亡命 亡命之徒 的 这个 同伙 里面 有 男人 ， 也 有 女...
3,<start> and some of them are not human be at a...,<start> 呃 还有 一众 亡命 亡命之徒 它 并 不是 人 。 <end>
4,<start> the daredevil are veri adventur as wel...,<start> 然后 亡命 亡命之徒 呢 总是 很 喜欢 冒险 ， 然后 他们 偶尔 也 比...


In [39]:
# tokenising
def tokenize(lang_data):
  tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', num_words=5000)
  tokenizer.fit_on_texts(lang_data)

  # word_index = tokenizer.word_index
  tensor = tokenizer.texts_to_sequences(lang_data)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
  return tensor, tokenizer

def load_dataset(dataframe):
  eng_tensor, eng_lang_tokenizer = tokenize(dataframe['English'])
  cn_tensor, cn_lang_tokenizer = tokenize(dataframe['Chinese'])
  return eng_tensor, cn_tensor, eng_lang_tokenizer, cn_lang_tokenizer

eng_tensor, cn_tensor, eng_lang_tokenizer, cn_lang_tokenizer = load_dataset(df)

# (Chinese -> English)
### Dataset

In [40]:
# Creating training and validation sets using an 80-20 split (Chinese -> English)
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(cn_tensor, eng_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

76 76 20 20


In [41]:
# Dataset Creationui
BUFFER_SIZE = len(input_tensor_train) # len of translations
BATCH_SIZE = 64
STEPS_PER_EPOCH = BUFFER_SIZE//BATCH_SIZE
EMBEDDING_DIM = 256 # output dims of embedding layer (dims of embedded obj)
ENCODER_DIM = 1024  #  output dims of GRU layer
VOCAB_INPUT_SIZE = len(cn_lang_tokenizer.word_index) + 1
VOCAB_OUTPUT_SIZE = len(eng_lang_tokenizer.word_index) + 1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape

<BatchDataset shapes: ((64, 65), (64, 49)), types: (tf.int32, tf.int32)>


TensorShape([64, 65])

In [42]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
    super(Encoder, self).__init__()
    self.batch_size = batch_size
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_size, self.enc_units))

In [43]:
encoder = Encoder(VOCAB_INPUT_SIZE, EMBEDDING_DIM, ENCODER_DIM, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder output shape: (batch size, sequence length, units) (64, 65, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [44]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # query hidden state shape == (batch_size, hidden size)
    # query_with_time_axis shape == (batch_size, 1, hidden size)
    # values shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    query_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [45]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 65, 1)


In [46]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(hidden, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)
    # print(f"hidden: {hidden.shape}")
    # print(f"output: {enc_output.shape}")
    # print(f"embedding: {x.shape}")
    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    # print(f"concat: {x.shape}")
    # passing the concatenated vector to the GRU
    output, state = self.gru(x)
    # print(f"gru: {output.shape}, {state.shape}")
    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))
    # print(f"reshape: {output.shape}")
    # output shape == (batch_size, vocab)
    x = self.fc(output)
    # print(f"FC: {x.shape}")
    return x, state, attention_weights

In [47]:
decoder = Decoder(VOCAB_OUTPUT_SIZE, EMBEDDING_DIM, ENCODER_DIM, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))
# hidden: (64, 1024)
# output: (64, 67, 1024)
# embedding: (64, 1, 256)
# concat: (64, 1, 1280)
# gru: (64, 1, 1024), (64, 1024)
# reshape: (64, 1024)
# FC: (64, 524)
# Decoder output shape: (batch_size, vocab size) (64, 524)

Decoder output shape: (batch_size, vocab size) (64, 508)


In [48]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [49]:
loss_function

<function __main__.loss_function(real, pred)>

In [50]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [51]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([eng_lang_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
      print(f"predictions: {predictions} // targ[:, t]: {targ[:, t]} // dec_hidden: {dec_hidden}")
      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [52]:
EPOCHS = 2

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(STEPS_PER_EPOCH)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

predictions: Tensor("decoder/dense_3/BiasAdd:0", shape=(64, 508), dtype=float32) // targ[:, t]: Tensor("strided_slice:0", shape=(64,), dtype=int32) // dec_hidden: Tensor("decoder/gru_1/StatefulPartitionedCall:2", shape=(64, 1024), dtype=float32)
predictions: Tensor("decoder_1/dense_3/BiasAdd:0", shape=(64, 508), dtype=float32) // targ[:, t]: Tensor("strided_slice_3:0", shape=(64,), dtype=int32) // dec_hidden: Tensor("decoder_1/gru_1/StatefulPartitionedCall:2", shape=(64, 1024), dtype=float32)
predictions: Tensor("decoder_2/dense_3/BiasAdd:0", shape=(64, 508), dtype=float32) // targ[:, t]: Tensor("strided_slice_6:0", shape=(64,), dtype=int32) // dec_hidden: Tensor("decoder_2/gru_1/StatefulPartitionedCall:2", shape=(64, 1024), dtype=float32)
predictions: Tensor("decoder_3/dense_3/BiasAdd:0", shape=(64, 508), dtype=float32) // targ[:, t]: Tensor("strided_slice_9:0", shape=(64,), dtype=int32) // dec_hidden: Tensor("decoder_3/gru_1/StatefulPartitionedCall:2", shape=(64, 1024), dtype=float32

predictions: Tensor("decoder_33/dense_3/BiasAdd:0", shape=(64, 508), dtype=float32) // targ[:, t]: Tensor("strided_slice_99:0", shape=(64,), dtype=int32) // dec_hidden: Tensor("decoder_33/gru_1/StatefulPartitionedCall:2", shape=(64, 1024), dtype=float32)
predictions: Tensor("decoder_34/dense_3/BiasAdd:0", shape=(64, 508), dtype=float32) // targ[:, t]: Tensor("strided_slice_102:0", shape=(64,), dtype=int32) // dec_hidden: Tensor("decoder_34/gru_1/StatefulPartitionedCall:2", shape=(64, 1024), dtype=float32)
predictions: Tensor("decoder_35/dense_3/BiasAdd:0", shape=(64, 508), dtype=float32) // targ[:, t]: Tensor("strided_slice_105:0", shape=(64,), dtype=int32) // dec_hidden: Tensor("decoder_35/gru_1/StatefulPartitionedCall:2", shape=(64, 1024), dtype=float32)
predictions: Tensor("decoder_36/dense_3/BiasAdd:0", shape=(64, 508), dtype=float32) // targ[:, t]: Tensor("strided_slice_108:0", shape=(64,), dtype=int32) // dec_hidden: Tensor("decoder_36/gru_1/StatefulPartitionedCall:2", shape=(64,

predictions: Tensor("decoder_18/dense_3/BiasAdd:0", shape=(64, 508), dtype=float32) // targ[:, t]: Tensor("strided_slice_54:0", shape=(64,), dtype=int32) // dec_hidden: Tensor("decoder_18/gru_1/StatefulPartitionedCall:2", shape=(64, 1024), dtype=float32)
predictions: Tensor("decoder_19/dense_3/BiasAdd:0", shape=(64, 508), dtype=float32) // targ[:, t]: Tensor("strided_slice_57:0", shape=(64,), dtype=int32) // dec_hidden: Tensor("decoder_19/gru_1/StatefulPartitionedCall:2", shape=(64, 1024), dtype=float32)
predictions: Tensor("decoder_20/dense_3/BiasAdd:0", shape=(64, 508), dtype=float32) // targ[:, t]: Tensor("strided_slice_60:0", shape=(64,), dtype=int32) // dec_hidden: Tensor("decoder_20/gru_1/StatefulPartitionedCall:2", shape=(64, 1024), dtype=float32)
predictions: Tensor("decoder_21/dense_3/BiasAdd:0", shape=(64, 508), dtype=float32) // targ[:, t]: Tensor("strided_slice_63:0", shape=(64,), dtype=int32) // dec_hidden: Tensor("decoder_21/gru_1/StatefulPartitionedCall:2", shape=(64, 10

NameError: name 'steps_per_epoch' is not defined

In [53]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = array_to_string([w for w in jieba.cut_for_search(sentence)])

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, enc_units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [54]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [55]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [56]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [57]:
translate('知识最快')

NameError: name 'inp_lang' is not defined

In [ ]:
inp_lang.word_index